Title: Analysis of Resting Blood Pressure vs. Maximum Heart Rate for Heart disease

For my project proposal, I will be using is a modified version of the UCI Heart Disease dataset. The UCI Heart Disease dataset includes four databases of the countries: Cleveland, Hungary, Switzerland, and the VA Long Beach. I will be using Cleveland for the prediction of heart disease because the dataframe contains sufficient observations. The Heart Disease Cleveland dataframe includes the variables: "trestbps" and "thalach", which I will be using to predict our variable of intrest: "num". In the dataframe, "trestbps" refers to the "resting blood pressure (mm Hg)" on admission to the hospital, while "thalach" refers to "maximum heart rate achieved (bps)", and "num" means the diagnosis of heart disease with a value of: 0 means no heart disease, 1-4 means the levels of heart disease[1] (We can classify levels of heart disease as 1 = Mild, 2 = Moderate, 3 = Severe 4 = Life threatening)[2]. With these resources, I will be answering the question: Can I classify the extent of heart disease using resting blood pressure and maximum heart rate achieved?.

[1]: Source from Piazza question @555

[2]: For the levels of heart disease, we refer to the document https://www.rigshospitalet.dk/afdelinger-og-klinikker/kraeft-og-organsygdomme/blodsygdomme/forskning/forsoegsbehandling/Documents/Lymfomer/Triangle/Triangle-SAE.pdf